# N3

In [2]:
def apply_n3_uncensoring(df):
    """
    N3: replace censored values with max(current value, average of non-censored values) within each POS.
    """
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()
    
    # Compute open group means and merge back
    open_means = (
        df[df['Zensiert'] == 0]
        .groupby('EHASTRA_EH_NUMMER')['Verkauf']
        .mean()
        .reset_index()
        .rename(columns={'Verkauf': 'open_mean'})
        .round()
    )
    
    df = df.merge(open_means, on='EHASTRA_EH_NUMMER', how='left')
    
    # CREATE MASK AFTER MERGE - this is the key fix
    mask = (df['Zensiert'] == 1) & df['open_mean'].notna()
    df.loc[mask, 'Verkauf_Uncensored'] = np.maximum(
        df.loc[mask, 'Verkauf'], 
        df.loc[mask, 'open_mean']
    )
    
    return df.drop('open_mean', axis=1)

# N2

In [3]:
def apply_n2_uncensoring(df):
    """
    N2: replace censored values with mean of uncensored values within each POS.
    """
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()
    
    # Calculate mean of uncensored observations for each group
    uncensored_means = (
        df[df['Zensiert'] == 0]
        .groupby(['EHASTRA_EH_NUMMER'])['Verkauf']
        .mean()
        .rename('uncensored_mean')
        .reset_index()
        .round()
    )
    
    # Merge back to original DataFrame
    df = df.merge(uncensored_means, on=['EHASTRA_EH_NUMMER'], how='left')
    
    # CREATE MASK AFTER MERGE - this is the key fix
    mask = (df['Zensiert'] == 1) & df['uncensored_mean'].notna()
    df.loc[mask, 'Verkauf_Uncensored'] = df.loc[mask, 'uncensored_mean']
    
    return df.drop('uncensored_mean', axis=1)

# N1

In [4]:
def apply_n1_uncensoring(df):
    """
    N1: replace censored values with mean of all values within each POS.
    """
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()

    # Identify closed observations
    is_closed = (df["Zensiert"] == 1)

    # Compute mean Verkauf per group and broadcast using transform
    group_means = df.groupby(['EHASTRA_EH_NUMMER'])['Verkauf_Uncensored'].transform('mean').round()
    
    # Replace closed observations
    df.loc[is_closed, 'Verkauf_Uncensored'] = group_means[is_closed]
    
    return df

# EM

In [6]:
def apply_em_uncensoring(df, max_iter=30, tolerance=1e-6):
    """    
    Parameters:
    - max_iter: Maximum number of iterations
    - tolerance: Convergence tolerance
    """
    
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()
    
    stockout_condition = (df['Zensiert'] == 1)
    
    for (pos,), group in df.groupby(['EHASTRA_EH_NUMMER']):
        try:
            group_stockout = stockout_condition.loc[group.index]
            
            if not group_stockout.any():
                continue
            
            sales = group['Verkauf_Uncensored'].values
            is_stockout = group_stockout.values
            
            uncensored = sales[~is_stockout]
            censored = sales[is_stockout]
            
            # Skip if no uncensored data - but keep original values
            if len(uncensored) == 0:
                continue
            
            # Quick lambda initialization
            lambda_est = np.mean(uncensored) if len(uncensored) > 0 else np.mean(sales) * 1.5
            lambda_est = max(lambda_est, 0.1)
            
            # Fast EM loop
            for iteration in range(max_iter):
                lambda_old = lambda_est
                
                # Batch E-step
                surv_prob = 1 - poisson.cdf(censored - 1, lambda_est)
                exact_prob = poisson.pmf(censored, lambda_est)
                surv_prob = np.maximum(surv_prob, 1e-12)
                
                expected = lambda_est + censored * exact_prob / surv_prob
                expected = np.maximum(expected, censored.astype(float))
                
                # M-step
                lambda_est = max(np.mean(np.concatenate([uncensored, expected])), 0.1)
                
                if abs(lambda_est - lambda_old) < tolerance:
                    break
            
            # Final update
            surv_prob = 1 - poisson.cdf(censored - 1, lambda_est)
            exact_prob = poisson.pmf(censored, lambda_est)
            surv_prob = np.maximum(surv_prob, 1e-12)
            
            final_expected = lambda_est + censored * exact_prob / surv_prob
            final_expected = np.maximum(final_expected, censored.astype(float))
            
            # Update original dataframe
            stockout_indices = group.index[is_stockout]
            df.loc[stockout_indices, 'Verkauf_Uncensored'] = final_expected.round()
            
        except Exception as e:
            print(f"EM error for POS {pos}: {e}")
            # Fill with original Verkauf values for this POS when error occurs
            group_stockout_indices = group.index[stockout_condition.loc[group.index]]
            df.loc[group_stockout_indices, 'Verkauf_Uncensored'] = df.loc[group_stockout_indices, 'Verkauf']
            continue
    
    # Replace any NaN values with original Verkauf
    nan_mask = df['Verkauf_Uncensored'].isna()
    df.loc[nan_mask, 'Verkauf_Uncensored'] = df.loc[nan_mask, 'Verkauf']
    
    return df

# PD

In [7]:
def apply_pd_uncensoring(df, tau=0.5, max_iter=20, tolerance=1e-4):
    """PD with skip for invalid POS groups and fallback to original Verkauf"""
    df = df.copy()
    df["is_closed"] = (df["Zensiert"] == 1)
    df['Verkauf_Uncensored'] = df['Verkauf'].copy().astype(float)

    def compute_pd_projection1(obs_val, lambda_est, tau):
        """
        Compute the PD projection for a single observation using Poisson distribution.
        This balances area A (original to new estimate) with area B (new estimate to infinity)
        weighted by parameter tau.
        """
        try:
            # Check for NaN inputs
            if pd.isna(obs_val) or pd.isna(lambda_est) or lambda_est <= 0:
                return float(obs_val) if not pd.isna(obs_val) else 0.0
            
            obs_val = int(round(obs_val))
            
            def objective(k_proj):
                k_proj = int(round(k_proj))
                
                if k_proj < obs_val:
                    return float('inf')
                
                # Area A: P(obs_val <= X <= k_proj)
                area_A = poisson.cdf(k_proj, lambda_est) - poisson.cdf(obs_val - 1, lambda_est)
                
                # Area B: P(X > k_proj)
                area_B = 1 - poisson.cdf(k_proj, lambda_est)
                
                if area_B > 1e-10: # Avoid division by zero
                    ratio = area_A / area_B
                    target_ratio = (1 - tau) / tau
                    return abs(ratio - target_ratio)
                else:
                    return abs(area_A - (1 - tau))
            
            # Search for optimal projection
            upper_bound = int(obs_val + max(10, int(3 * np.sqrt(lambda_est))))
            
            best_k = obs_val
            best_objective = float('inf')
            
            # Discrete search
            for k in range(int(obs_val), upper_bound + 1):
                obj_val = objective(k)
                if obj_val < best_objective:
                    best_objective = obj_val
                    best_k = k
            
            return float(best_k)
            
        except Exception:
            # Fallback to original value
            return float(obs_val) if not pd.isna(obs_val) else 0.0
    
    # Process groups
    grouped = df.groupby('EHASTRA_EH_NUMMER')
    
    for pos, group in grouped:
        try:
            open_mask = ~group['is_closed']
            closed_mask = group['is_closed']
            
            open_sales = group.loc[open_mask, 'Verkauf_Uncensored'].values
            closed_sales = group.loc[closed_mask, 'Verkauf_Uncensored'].values
            
            # SKIP POS WITHOUT VALID UNCENSORED DATA
            if len(open_sales) == 0 or np.all(pd.isna(open_sales)):
                # print(f"Skipping POS {pos}: no valid uncensored data")
                continue
            
            # Initialize lambda parameter
            lambda_est = np.mean(open_sales[~pd.isna(open_sales)])
            
            # SKIP IF LAMBDA IS INVALID
            if pd.isna(lambda_est) or lambda_est <= 0:
                print(f"Skipping POS {pos}: invalid lambda {lambda_est}")
                continue
            
            lambda_est = max(lambda_est, 0.1)
            closed_indices = group[closed_mask].index.values
            
            # Iterative process
            for iteration in range(max_iter):
                lambda_old = lambda_est
                
                # Project closed observations
                projected_values = np.array([
                    compute_pd_projection1(obs, lambda_est, tau) 
                    for obs in closed_sales
                ])
                
                # Re-estimate lambda
                all_values = np.concatenate([open_sales, projected_values])
                lambda_est = np.mean(all_values)
                lambda_est = max(lambda_est, 0.1)
                
                # Check convergence
                if abs(lambda_est - lambda_old) < tolerance:
                    break
            
            # Final projection
            final_projections = np.array([
                compute_pd_projection1(obs, lambda_est, tau) 
                for obs in closed_sales
            ])
            
            # Update dataframe
            df.loc[closed_indices, 'Verkauf_Uncensored'] = final_projections.round()
            
        except Exception as e:
            print(f"PD error for POS {pos}: {e}")
            # FALLBACK: Keep original values for this POS
            continue
    
    # FINAL FALLBACK: Any remaining NaN values get original Verkauf
    nan_mask = df['Verkauf_Uncensored'].isna()
    df.loc[nan_mask, 'Verkauf_Uncensored'] = df.loc[nan_mask, 'Verkauf']
    
    return df.drop('is_closed', axis=1)

# Conrad

In [8]:
from scipy.stats import poisson
import pandas as pd
import numpy as np

def berechnung(links, rechts, n, N, r, x_summe, value_tol=0.00001, max_iterations=1000):
    """
    Till's Code
    """
    iteration = 0
    
    while iteration < max_iterations:
        mu = (links + rechts) / 2
        wert_0 = (x_summe - mu * n) * (1 - poisson.cdf(N-1, mu)) + mu * (n - r) * (1 - poisson.cdf(N-2, mu))
        
        if iteration < 3:
            print(f"Iter {iteration}: mu={mu:.4f}, wert_0={wert_0:.8f}")
        
        if abs(wert_0) < value_tol:
            print(f"Converged after {iteration} iterations: mu={mu:.4f}, wert_0={wert_0:.8f}")
            return mu
        elif wert_0 < 0:  
            rechts = mu
        elif wert_0 > 0:
            links = mu
            
        iteration += 1
    
    print(f"✗ Max iterations reached: mu={mu:.4f}")
    return mu

def test_conrad_example():
    links = 1
    rechts = 100
    n = 13
    N = 10
    r = 7
    x_summe = 58
    
    print(f"links={links}, rechts={rechts}")
    print(f"n={n}, N={N}, r={r}, x_summe={x_summe}")
    
    result = berechnung(links, rechts, n, N, r, x_summe)
    print(f"Result: μ = {result:.4f}")
    print(f"Expected: μ ≈ 10.18")
    
    return result

def create_order_specific_mu_dict(df):

    order_specific_mu_dict = {}
    
    for (year, week), week_data in df.groupby(['Heftjahr', 'Heftnummer']):
        for bezug_val, group in week_data.groupby('Bezug'):
            n = len(group)
            N = bezug_val
            
            # Count non-stockouts
            stockouts_mask = (group['Zensiert'] == 1)
            r = n - stockouts_mask.sum()  # r = number of NON-stockouts
            
            # x_summe = sum of UNCENSORED observations only
            uncensored_sales = group[~stockouts_mask]['Verkauf']
            x_summe = uncensored_sales.sum()
            
            # Skip problematic cases
            if n < 3:
                continue
            if r == n:  # No stockouts = no censoring information
                continue
            if r == 0:  # All stockouts = no uncensored observations
                continue
            
            try:
                links = 1
                rechts = 100
                mu_est = berechnung(links, rechts, n, N, r, x_summe)
                if mu_est:
                    key = (year, week, bezug_val)
                    order_specific_mu_dict[key] = mu_est
            except Exception as e:
                print(f"Error in week {week}, Bezug {N}: {e}")
                continue

    print(f"Successfully estimated μ for {len(order_specific_mu_dict)} groups")
    return order_specific_mu_dict

def expected_poisson_tail(mu, N, max_k=200):
    """
    Compute E[X | X >= N] for X ~ Poisson(mu)
    """
    k_vals = np.arange(N, max_k)
    pmf = poisson.pmf(k_vals, mu)
    tail_prob = 1 - poisson.cdf(N - 1, mu)
    if tail_prob < 1e-8:
        return N  # fallback: don't uncensor
    return np.sum(k_vals * pmf) / tail_prob

# Internal function 
def apply_conrad_uncensoring_1(df, order_specific_mu_dict):
    """
    Given a DataFrame with Verkauf, Bezug, is_stockout, Heftjahr, Heftnummer,
    replace Verkauf with E[X | X >= Bezug] when censored.
    """
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()

    for idx, row in df.iterrows():
        # Skip if no stockout occurred
        if row['Zensiert'] == 0:
            continue
            
        # Get the order-quantity-specific demand parameter
        key = (row['Heftjahr'], row['Heftnummer'], row['Bezug'])
        mu = order_specific_mu_dict.get(key, None)

        if mu is None:
            # no estimate available for this specific (week, order_quantity) — keep original value
            continue

        # POS sold out — uncensor using the specific distribution for this order quantity
        est_demand = expected_poisson_tail(mu, row['Bezug'])
        df.at[idx, 'Verkauf_Uncensored'] = np.round(est_demand)

    return df

# Wrapper function for script
def apply_conrad_uncensoring(df):
    """
    WRAPPER FUNCTION: This is what gets called by the main processing loop
    """
    # Step 1: Create mu dictionary from the dataset
    order_specific_mu_dict = create_order_specific_mu_dict(df)
    
    # Step 2: Apply uncensoring using the mu dictionary  
    return apply_conrad_uncensoring_1(df, order_specific_mu_dict)

# print("\nRunning on actual data:")
# order_specific_mu_dict = create_order_specific_mu_dict(df)
# df_conrad_uncensored = apply_conrad_uncensoring(df, order_specific_mu_dict)

# Nahmias

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def nahmias_estimation(sales, S):
    """
    Nahmias method for censored normal data
    """
    sales = np.array(sales)
    n = len(sales)
    
    observed = sales[sales < S]
    r = len(observed)
    p = r / n
    
    if r < 2 or r >= n-1 or p <= 0 or p >= 1:
        return None, None
    
    try:
        x_bar = np.mean(observed)
        s2 = np.var(observed, ddof=1)
        z = norm.ppf(p)
        
        sigma_hat2 = s2 / (1 - (z * norm.pdf(z) / p) - (norm.pdf(z)**2 / p**2))
        sigma_hat = np.sqrt(sigma_hat2)
        mu_hat = x_bar + sigma_hat * norm.pdf(z) / p
        
        if not np.isfinite(mu_hat) or not np.isfinite(sigma_hat) or sigma_hat <= 0:
            return None, None
            
        return mu_hat, sigma_hat
        
    except Exception:
        return None, None

def create_order_specific_nahmias_dict(df):
    """
    Create μ and σ estimates for each (week, order_quantity) combination
    """
    order_specific_mu_dict = {}
    order_specific_sigma_dict = {}

    for (year, week), week_data in df.groupby(['Heftjahr', 'Heftnummer']):
        for bezug_val, group in week_data.groupby('Bezug'):
            n = len(group)
            S = bezug_val
            
            if n < 5:
                continue
            
            sales = group['Verkauf'].values
            
            try:
                mu_est, sigma_est = nahmias_estimation(sales, S)
                if mu_est is not None and sigma_est is not None:
                    key = (year, week, bezug_val)
                    order_specific_mu_dict[key] = mu_est
                    order_specific_sigma_dict[key] = sigma_est
            except Exception as e:
                continue

    # print(f"Successfully estimated μ,σ for {len(order_specific_mu_dict)} groups")
    return order_specific_mu_dict, order_specific_sigma_dict

def expected_normal_tail(mu, sigma, S):
    """
    Compute E[X | X >= S] for X ~ Normal(mu, sigma)
    """
    if sigma <= 0:
        return S
    
    z = (S - mu) / sigma
    
    if z > 6:
        return S
    
    tail_prob = 1 - norm.cdf(z)
    
    if tail_prob < 1e-10:
        return S
    
    expected_value = mu + sigma * norm.pdf(z) / tail_prob
    
    return expected_value

def apply_nahmias_uncensoring_1(df, order_specific_mu_dict, order_specific_sigma_dict):
    """
    Uncensor dataset using Nahmias estimates
    """
    df = df.copy()
    df['Verkauf_Uncensored'] = df['Verkauf'].copy()

    for idx, row in df.iterrows():
        if row['Zensiert'] == 0:
            continue
            
        key = (row['Heftjahr'], row['Heftnummer'], row['Bezug'])
        mu = order_specific_mu_dict.get(key, None)
        sigma = order_specific_sigma_dict.get(key, None)

        if mu is None or sigma is None:
            continue

        est_demand = expected_normal_tail(mu, sigma, row['Bezug'])
        df.at[idx, 'Verkauf_Uncensored'] = np.round(est_demand)

    return df

def test_nahmias():
    """Test implementation"""
    mu_true = 100
    sigma_true = 30
    S = 110
    n = 100
    
    np.random.seed(42)
    demand = np.random.normal(mu_true, sigma_true, n)
    sales = np.minimum(demand, S)
    
    print("Testing Nahmias implementation:")
    print(f"True μ: {mu_true}, True σ: {sigma_true}")
    print(f"S (censoring limit): {S}")
    print(f"Sample size: {n}")
    
    mu_hat, sigma_hat = nahmias_estimation(sales, S)
    
    naive_mean = np.mean(sales)
    naive_std = np.std(sales, ddof=1)
    
    print(f"True mean: {mu_true}")
    print(f"Naive mean (sales): {naive_mean:.2f}")
    print(f"Corrected estimator (Nahmias): {mu_hat:.2f}")
    print(f"True Std.Dev.: {sigma_true}")
    print(f"Corrected Std.Dev.: {sigma_hat:.2f}")
    print(f"Naive Std.Dev. (sales): {naive_std:.2f}")
    
    return mu_hat, sigma_hat

def apply_nahmias_uncensoring(df):
    """
    WRAPPER FUNCTION: This is what gets called by the main processing loop
    """
    # Step 1: Create mu dictionary from the dataset
    order_specific_mu_dict, compute_mu_sigma_nahmias = create_order_specific_nahmias_dict(df)
    
    # Step 2: Apply uncensoring using the mu dictionary  
    return apply_nahmias_uncensoring_1(df, order_specific_mu_dict, compute_mu_sigma_nahmias)

# Baseline

In [10]:
def apply_baseline_uncensoring(df):
    df_result = df.copy()
    df_result['Verkauf_Uncensored'] = df_result['Verkauf'] 
    return df_result

# Script for creating csv of all features and optimization KPIs for all mags

Loops through all magazines and applies uncensoring methods (n1,n2,n3,em,pd,conrad,nahmias), evaluates optimization KPIs

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path

def build_empirical(df_counts):
    '''
    Given a value counts dataframe, return a dataframe indexed by demand and containing columns count, pmf value, and cmf value
    '''
    demand_distribution = pd.DataFrame(df_counts)
    demand_distribution.sort_index(inplace=True)
    demand_distribution['pmf'] = demand_distribution['count']/demand_distribution['count'].sum()
    demand_distribution['cmf'] = demand_distribution['pmf'].cumsum()
    return demand_distribution

def update_empirical(empirical, demand):
    '''
    Update empirical demand distribution with new uncensored demand
    '''
    # Safety check: ensure empirical is not empty
    if empirical.empty:
        # Create new empirical from this single observation
        empirical = build_empirical(pd.Series([demand]).value_counts())
        return empirical
    
    if demand in empirical.index: 
        empirical.loc[demand, 'count'] += 1
    else:
        empirical.loc[demand, 'count'] = 1
        empirical.sort_index(inplace=True)
    
    # Recalculate PMF and CMF
    total_count = empirical['count'].sum()
    if total_count > 0:
        empirical['pmf'] = empirical['count'] / total_count
        empirical['cmf'] = empirical['pmf'].cumsum()
        
    return empirical

def calculate_optimal_inventory(empirical, service_level=0.9):
    '''
    Calculate optimal inventory using newsvendor model with safety checks
    '''
    if empirical.empty or len(empirical) == 0:
        return 0  # Default to 0 if no data
    
    candidates = (empirical['cmf'] >= service_level)
    if not candidates.any():
        # If no demand level meets service level, return highest demand
        return empirical.index.max()
    
    return candidates.idxmax()

def calculate_pinball_loss(q_ordered, demand_actual, alpha=0.9):
    '''
    Calculate pinball loss for a single observation
    alpha: service level (e.g., 0.9 for 90% service level)
    '''
    if q_ordered >= demand_actual:
        # Overstock cost
        return (1 - alpha) * (q_ordered - demand_actual)
    else:
        # Understock cost
        return alpha * (demand_actual - q_ordered)

def calculate_inventory_kpis_for_pos(pos_data, method_name, pos_id):
    """
    Calculate inventory KPIs for a single POS
    
    Returns:
    - pinball_loss: Average pinball loss across all periods
    - oos_rate_by_hf: Out-of-stock rate for each Heftnummer
    - alpha_service_level_by_hf: Alpha service level for each Heftnummer  
    - beta_fill_rate_by_hf: Beta fill rate for each Heftnummer
    """
    
    if len(pos_data) == 0:
        return {
            'pinball_loss': 0,
            'avg_oos_rate': 0,
            'avg_alpha_service_level': 0,
            'avg_beta_fill_rate': 0,
        }
    
    # Calculate pinball loss for each period
    pinball_losses = []
    for _, row in pos_data.iterrows():
        q_ordered = row[f'{method_name}_Q']
        demand_actual = row['Verkauf_MBR']
        if pd.notna(q_ordered) and pd.notna(demand_actual):
            loss = calculate_pinball_loss(q_ordered, demand_actual, alpha=0.9)
            pinball_losses.append(loss)
    
    avg_pinball_loss = np.mean(pinball_losses) if pinball_losses else 0
    
    # Calculate KPIs by Heftnummer (HF)
    hf_kpis = []
    
    for hf, hf_data in pos_data.groupby('Heftnummer'):
        if len(hf_data) == 0:
            continue
            
        # Out-of-stock events (demand > ordered quantity)
        oos_events = (hf_data['Verkauf_MBR'] > hf_data[f'{method_name}_Q']).sum()
        total_periods = len(hf_data)
        
        # OOS Rate
        oos_rate = oos_events / total_periods if total_periods > 0 else 0
        
        # Alpha Service Level (1 - OOS Rate)
        alpha_service_level = 1 - oos_rate
        
        # Beta Fill Rate (units satisfied / total demand)
        total_demand = hf_data['Verkauf_MBR'].sum()
        units_satisfied = np.minimum(hf_data[f'{method_name}_Q'], hf_data['Verkauf_MBR']).sum()
        beta_fill_rate = units_satisfied / total_demand if total_demand > 0 else 1
        
        hf_kpis.append({
            'heftnummer': hf,
            'oos_rate': oos_rate,
            'alpha_service_level': alpha_service_level,
            'beta_fill_rate': beta_fill_rate
        })
    
    # Average across all Heftnummers
    if hf_kpis:
        avg_oos_rate = np.mean([kpi['oos_rate'] for kpi in hf_kpis])
        avg_alpha_service_level = np.mean([kpi['alpha_service_level'] for kpi in hf_kpis])
        avg_beta_fill_rate = np.mean([kpi['beta_fill_rate'] for kpi in hf_kpis])
    else:
        avg_oos_rate = 0
        avg_alpha_service_level = 0
        avg_beta_fill_rate = 0
    
    return {
        'pinball_loss': avg_pinball_loss,
        'avg_oos_rate': avg_oos_rate,
        'avg_alpha_service_level': avg_alpha_service_level,
        'avg_beta_fill_rate': avg_beta_fill_rate,
    }

def generate_inventory_recommendations(df, method_name):
    """
    Generate inventory recommendations for all POS using empirical demand distribution
    """
    inventory_recommendations = []
    
    for pos_id, pos_group in df.groupby('EHASTRA_EH_NUMMER'):
        try:
            # Step 1: Initialize empirical demand distribution from 2022-2023 data
            df_train = pos_group[pos_group['Heftjahr'] < 2024]
            if df_train.empty: 
                continue
                
            # Use the uncensored demand column (standardized across all methods)
            if 'Verkauf_Uncensored' not in df_train.columns:
                continue
            
            # Get valid uncensored values for training
            uncensored_values = df_train['Verkauf_Uncensored'].dropna()
            
            if len(uncensored_values) == 0:
                continue
                
            # Check for all-zero demand case
            if (uncensored_values == 0).all():
                # Create minimal distribution with 0 and 1
                empirical = build_empirical(pd.Series([0, 1]).value_counts())
            else:
                value_counts = uncensored_values.value_counts()
                empirical = build_empirical(value_counts)
            
            # Step 2: Use newsvendor model to calculate optimal inventory for 2024
            df_test = pos_group[pos_group['Heftjahr'] == 2024]
            df_test = df_test.sort_values(['Period'])
            
            for _, row in df_test.iterrows():
                # Calculate optimal order quantity
                optimal_q = calculate_optimal_inventory(empirical, service_level=0.9)
                
                inventory_recommendations.append({
                    'EHASTRA_EH_NUMMER': pos_id,
                    'Period': row.Period,
                    'Heftnummer': row.Heftnummer if 'Heftnummer' in row else None,
                    'Verkauf_MBR': row.Verkauf_MBR if 'Verkauf_MBR' in row else None,
                    f'{method_name}_Q': optimal_q
                })
                
                # Update empirical distribution with new observation
                if pd.notna(row['Verkauf_Uncensored']) and row['Verkauf_Uncensored'] >= 0:
                    empirical = update_empirical(empirical, row['Verkauf_Uncensored'])
                    
        except Exception as e:
            # Only print errors, not debug info
            print(f"    Error in POS {pos_id}: {str(e)}")
            continue
    
    return pd.DataFrame(inventory_recommendations)

def parse_filename(filename):
    """
    Parse filename to extract magazine letter and parameters
    """
    parts = filename.split('_')
    if len(parts) >= 4:
        magazine = parts[0]
        date = parts[1]
        zq = parts[2]
        zg = parts[3]
        return magazine, date, zq, zg
    return None, None, None, None

def process_inventory_optimization(data_directory, output_filename='inventory_optimization_results_wide.csv'):
    """
    Process all datasets and generate inventory optimization results in wide format
    where each row is a unique EHASTRA_EH_NUMMER + Magazine combination,
    and columns contain KPIs for each method (e.g., N1_pinball_loss, N2_pinball_loss, etc.)
    """
    
    # Method mapping to function names (you'll need to import these functions)
    method_functions = {
        'N1': apply_n1_uncensoring,
        'N2': apply_n2_uncensoring,
        'N3': apply_n3_uncensoring,
        'EM': apply_em_uncensoring,
        'PD': apply_pd_uncensoring,
        'Nahmias': apply_nahmias_uncensoring,
        'Conrad': apply_conrad_uncensoring,
        'Baseline': apply_baseline_uncensoring
    }
    
    # Find original files with ZQ0.35_ZG0.4 pattern (without method names)
    pattern = os.path.join(data_directory, "*_20250212_ZQ0.35_ZG0.4_testfile.csv")
    csv_files = glob.glob(pattern)
    
    if not csv_files:
        print(f"No original CSV files found matching pattern *_20250212_ZQ0.35_ZG0.4_testfile.csv in {data_directory}")
        return
    
    print(f"Found {len(csv_files)} original CSV files to process")
    
    # Dictionary to store results by POS + Magazine combination
    results_by_pos_magazine = {}
    
    for csv_file in csv_files:
        filename = os.path.basename(csv_file)
        magazine, date, zq, zg = parse_filename(filename)
        
        if not magazine:
            print(f"Could not parse filename: {filename}")
            continue
            
        print(f"Processing {filename}...")
        
        try:
            # Read the original CSV file
            original_df = pd.read_csv(csv_file)
            
            # Filter out POS locations with insufficient data
            pos_counts = original_df.groupby('EHASTRA_EH_NUMMER').size()
            valid_pos_ids = pos_counts[pos_counts >= 3].index
            original_df = original_df[original_df['EHASTRA_EH_NUMMER'].isin(valid_pos_ids)]
            
            if len(original_df) == 0:
                print(f"  No valid POS locations in {filename}")
                continue
            
            # Filter to keep only POS that exist in both training (2022-2023) and test (2024) periods
            pos_in_train = original_df[original_df['Heftjahr'] < 2024]['EHASTRA_EH_NUMMER'].unique()
            pos_in_test = original_df[original_df['Heftjahr'] == 2024]['EHASTRA_EH_NUMMER'].unique()
            valid_pos_both_periods = set(pos_in_train) & set(pos_in_test)
            
            original_df = original_df[original_df['EHASTRA_EH_NUMMER'].isin(valid_pos_both_periods)]
            
            if len(original_df) == 0:
                print(f"  No POS locations exist in both training and test periods")
                continue
            
            print(f"  Found {len(valid_pos_both_periods)} POS locations with both training and test data")
            
            # Process each uncensoring method
            for method_name, method_function in method_functions.items():
                print(f"  Applying {method_name} uncensoring...")
                
                try:
                    # Apply the uncensoring method to get Verkauf_Uncensored column
                    uncensored_df = method_function(original_df.copy())
                    
                    # Handle NaN values in Verkauf_Uncensored by imputing with original Verkauf
                    nan_mask = uncensored_df['Verkauf_Uncensored'].isna()
                    uncensored_df.loc[nan_mask, 'Verkauf_Uncensored'] = uncensored_df.loc[nan_mask, 'Verkauf']
                    
                    # Generate inventory recommendations using uncensored data
                    inventory_df = generate_inventory_recommendations(uncensored_df, method_name)
                    
                    if inventory_df.empty:
                        print(f"    No inventory recommendations generated for {method_name}")
                        continue
                    
                    # Calculate KPIs for each POS
                    for pos_id, pos_data in inventory_df.groupby('EHASTRA_EH_NUMMER'):
                        kpis = calculate_inventory_kpis_for_pos(pos_data, method_name, pos_id)
                        
                        # Create unique key for POS + Magazine
                        pos_mag_key = (pos_id, magazine)
                        
                        # Initialize the result dictionary for this POS+Magazine if it doesn't exist
                        if pos_mag_key not in results_by_pos_magazine:
                            results_by_pos_magazine[pos_mag_key] = {
                                'EHASTRA_EH_NUMMER': pos_id,
                                'magazine': magazine,
                                'date': date,
                                'zq_parameter': zq,
                                'zg_parameter': zg,
                                'filename': filename
                            }
                        
                        # Add method-specific KPIs
                        for kpi_name, kpi_value in kpis.items():
                            #if kpi_name != 'n_periods':
                            column_name = f"{method_name}_{kpi_name}"
                            results_by_pos_magazine[pos_mag_key][column_name] = kpi_value
                        
                except Exception as e:
                    print(f"    Error applying {method_name}: {str(e)}")
                    continue
                
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue
    
    if results_by_pos_magazine:
        # Convert dictionary to DataFrame
        final_df = pd.DataFrame.from_dict(results_by_pos_magazine, orient='index')
        
        # Reset index since we used tuples as keys
        final_df.reset_index(drop=True, inplace=True)
        
        # Save to CSV
        output_path = output_filename 
        final_df.to_csv(output_path, index=False)
        
        print(f"\nInventory optimization processing complete!")
        print(f"Results saved as: {output_path}")
        print(f"Total rows: {len(final_df)}")
        print(f"Magazines processed: {final_df['magazine'].nunique()}")
        print(f"POS locations: {final_df['EHASTRA_EH_NUMMER'].nunique()}")
        
        # Print column names to show the wide format structure
        kpi_columns = [col for col in final_df.columns if any(method in col for method in method_functions.keys())]
        print(f"Method-specific KPI columns created: {len(kpi_columns)}")
        print(f"Sample KPI columns: {kpi_columns[:10]}")  # Show first 10 columns
        
        # Print summary statistics for one method as example
        if f'N1_pinball_loss' in final_df.columns:
            print(f"\nSample statistics (N1 method):")
            print(f"Average N1 Pinball Loss: {final_df['N1_pinball_loss'].mean():.4f}")
            print(f"Average N1 OOS Rate: {final_df['N1_avg_oos_rate'].mean():.4f}")
        
        return final_df
    else:
        print("No data was processed successfully.")
        return None

# Usage
if __name__ == "__main__":
    data_directory = "Test1"  # Change this to your actual directory path
    result_df = process_inventory_optimization(
        data_directory, 
        'NaiveEMPDConradNahmias_Optimization_KPIs_FeaturesNOTFINAL.csv'
    )

Found 9 original CSV files to process
Processing E_20250212_ZQ0.35_ZG0.4_testfile.csv...
  Found 3152 POS locations with both training and test data
  Applying N1 uncensoring...
  Applying N2 uncensoring...
  Applying N3 uncensoring...
  Applying EM uncensoring...
  Applying PD uncensoring...
Skipping POS EHA0231386: invalid lambda 0.0
Skipping POS EHA0746295: invalid lambda 0.0
Skipping POS EHA1036501: invalid lambda 0.0
Skipping POS EHA1177140: invalid lambda 0.0
Skipping POS EHA1266585: invalid lambda 0.0
Skipping POS EHA1490641: invalid lambda 0.0
Skipping POS EHA1603545: invalid lambda 0.0
Skipping POS EHA1765650: invalid lambda 0.0
Skipping POS EHA1928265: invalid lambda 0.0
Skipping POS EHA1938990: invalid lambda 0.0
Skipping POS EHA2425406: invalid lambda 0.0
Skipping POS EHA2876191: invalid lambda 0.0
Skipping POS EHA3557085: invalid lambda 0.0
Skipping POS EHA3946814: invalid lambda 0.0
Skipping POS EHA4243527: invalid lambda 0.0
Skipping POS EHA4376715: invalid lambda 0.0
Sk

# Adding best method

In [13]:
import pandas as pd
import numpy as np

def add_best_method_column(df):
    """
    Add a column showing which method performs best based on pinball loss
    with alpha service level as tiebreaker.
    
    Parameters:
    - df: DataFrame with wide format results (one row per POS+Magazine)
    
    Returns:
    - DataFrame with new column 'best_method_pinball_loss'
    """
    
    # Find all pinball_loss columns
    pinball_loss_cols = [col for col in df.columns if col.endswith('_pinball_loss')]
    alpha_service_cols = [col for col in df.columns if col.endswith('_avg_alpha_service_level')]
    
    if not pinball_loss_cols:
        raise ValueError("No pinball_loss columns found in DataFrame")
    
    print(f"Found {len(pinball_loss_cols)} methods with pinball_loss data")
    
    df_copy = df.copy()
    best_methods = []
    
    # For each row, find the method with lowest pinball loss
    for idx, row in df_copy.iterrows():
        # Get pinball loss values for all methods
        method_pinball_loss = {}
        for col in pinball_loss_cols:
            method_name = col.replace('_pinball_loss', '')  # Extract method name (e.g., 'N1' from 'N1_pinball_loss')
            if pd.notna(row[col]):
                method_pinball_loss[method_name] = row[col]
        
        if not method_pinball_loss:
            best_methods.append(None)
            continue
        
        # Find the method(s) with minimum pinball loss
        min_pinball_loss = min(method_pinball_loss.values())
        best_methods_pinball = [method for method, loss in method_pinball_loss.items() if loss == min_pinball_loss]
        
        # If there's only one best method, use it
        if len(best_methods_pinball) == 1:
            best_method = best_methods_pinball[0]
        else:
            # Use alpha service level as tiebreaker (higher is better)
            tiebreaker_values = {}
            for method in best_methods_pinball:
                alpha_col = f"{method}_avg_alpha_service_level"
                if alpha_col in df.columns and pd.notna(row[alpha_col]):
                    tiebreaker_values[method] = row[alpha_col]
            
            if not tiebreaker_values:
                # If no tiebreaker values available, pick the first method alphabetically
                best_method = sorted(best_methods_pinball)[0]
            else:
                # Pick method with highest alpha service level
                max_alpha = max(tiebreaker_values.values())
                best_method = [method for method, alpha in tiebreaker_values.items() if alpha == max_alpha][0]
        
        best_methods.append(best_method)
    
    # Add the best method column
    df_copy['best_method_pinball_loss'] = best_methods
    
    return df_copy

# Usage example
if __name__ == "__main__":
    # Load your data
    df = pd.read_csv("NaiveEMPDConradNahmias_Optimization_KPIs_FeaturesNOTFINAL.csv")
    
    # Add best method column
    df_with_best = add_best_method_column(df)
    
    # Analyze results
    print("\nBest method distribution:")
    print(df_with_best['best_method_pinball_loss'].value_counts())
    
    # Save results
    df_with_best.to_csv("all_methods_optimization_KPIs_Features_with_BestNOTFINAL.csv", index=False)
    print(f"\nResults saved with {len(df_with_best)} rows")

Found 8 methods with pinball_loss data

Best method distribution:
best_method_pinball_loss
N1          17040
N3           1665
EM            731
Conrad        688
Nahmias       168
N2             83
PD             32
Baseline        2
Name: count, dtype: int64

Results saved with 20409 rows


# Merging Sofortremission

In [14]:
import pandas as pd

# Read the files
best_methods_df = pd.read_csv("all_methods_optimization_KPIs_Features_with_BestNOTFINAL.csv")
sofortremission_df = pd.read_csv("sofortremissionFeatures3007.csv")

# Fix the EHASTRA_EH_NUMMER format - add "EH" prefix to sofortremission
sofortremission_df['EHASTRA_EH_NUMMER'] = 'EH' + sofortremission_df['EHASTRA_EH_NUMMER'].astype(str)

# Merge
merged_df = best_methods_df.merge(
    sofortremission_df, 
    left_on=['EHASTRA_EH_NUMMER', 'magazine'], 
    right_on=['EHASTRA_EH_NUMMER', 'VDZ'], 
    how='left'
).drop('VDZ', axis=1)

# Sort by magazine
merged_df = merged_df.sort_values('magazine')

# Save
merged_df.to_csv("Sales_DT_Input.csv", index=False)